In [1]:
%load_ext autoreload
%autoreload 2
import os
print("HF_HOME:", os.getenv("HF_HOME"))
print("HF_DATASETS_CACHE:", os.getenv("HF_DATASETS_CACHE"))
print("TRANSFORMERS_CACHE:", os.getenv("TRANSFORMERS_CACHE"))

from datasets import load_dataset, load_from_disk, Dataset as HFDataset
from transformers import AutoTokenizer, default_data_collator, AutoModelForCausalLM, AutoModel
from transformers.utils.import_utils import clear_import_cache
import torch
from torch import nn


HF_HOME: /media/mohamed/ssdnod/huggingface
HF_DATASETS_CACHE: /media/mohamed/ssdnod/huggingface/datasets
TRANSFORMERS_CACHE: /media/mohamed/ssdnod/huggingface/hub


/home/mohamed/miniconda3/envs/llm_wm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mohamed/miniconda3/envs/llm_wm/lib/python3.9/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:
import numpy as np 


hidden_size = 3072
LLM_hidden_Size = 768

X = torch.randint(0, 100, size=(LLM_hidden_Size, LLM_hidden_Size), dtype=torch.float32)

class IdTest(nn.Module):
    def __init__(self, LLM_hidden_Size, hidden_size):
        super().__init__()
        self.l1 = nn.Linear(LLM_hidden_Size, hidden_size, bias=False)
        W1 = torch.zeros((hidden_size, LLM_hidden_Size), dtype=torch.float32)
        W1[:LLM_hidden_Size, :LLM_hidden_Size] = torch.eye(LLM_hidden_Size)
        self.l1.weight.data = W1

        self.l2 = nn.Linear(hidden_size, LLM_hidden_Size, bias=False)
        W2 = torch.zeros((LLM_hidden_Size, hidden_size), dtype=torch.float32)
        W2[:LLM_hidden_Size, :LLM_hidden_Size] = torch.eye(LLM_hidden_Size)
        self.l2.weight.data = W2
    
    def forward(self, x):
        return self.l2(self.l1(x))

idtest = IdTest(LLM_hidden_Size, hidden_size)

with torch.no_grad():
    print(idtest.l1.weight)
    print(idtest.l2.weight)
    out = idtest(X)

# np.all(X == (X@(W1.T))@(W2.T))
torch.all(X==out)

Parameter containing:
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)
Parameter containing:
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)


tensor(True)

# Test with gpt2

In [4]:
import inspect
gpt2_model = AutoModelForCausalLM.from_pretrained("gpt2")
print(inspect.signature(gpt2_model.transformer.h[0].forward))


(hidden_states: Optional[tuple[torch.FloatTensor]], past_key_value: Optional[transformers.cache_utils.Cache] = None, cache_position: Optional[torch.LongTensor] = None, attention_mask: Optional[torch.FloatTensor] = None, head_mask: Optional[torch.FloatTensor] = None, encoder_hidden_states: Optional[torch.Tensor] = None, encoder_attention_mask: Optional[torch.FloatTensor] = None, use_cache: Optional[bool] = False, output_attentions: Optional[bool] = False, **kwargs) -> Union[tuple[torch.Tensor], tuple[torch.Tensor, tuple[torch.FloatTensor, ...]], NoneType]


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# model_name = "HuggingFaceTB/SmolLM3-3B"
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",         # Accelerate does the placement
    torch_dtype=torch.float16, # halves VRAM
)

# prompt = "Give me a brief explanation of gravity in simple terms."
# text = tokenizer.apply_chat_template(
#     [{"role": "user", "content": prompt}],
#     add_generation_prompt=True,
#     tokenize=False,
# )
# inputs = tokenizer([text], return_tensors="pt").to(model.device)

# out = model.generate(**inputs, max_new_tokens=256)
# print(tokenizer.decode(out[0][inputs.input_ids.shape[-1]:],
#                        skip_special_tokens=True))


In [8]:
count = 0
for p in model.parameters():
    count += p.numel()
print(f'total param count : {count/1e6:.3f} M')

total param count : 124.440 M


In [19]:
d= dict(model.named_modules())
d.keys()

dict_keys(['', 'transformer', 'transformer.wte', 'transformer.wpe', 'transformer.drop', 'transformer.h', 'transformer.h.0', 'transformer.h.0.ln_1', 'transformer.h.0.attn', 'transformer.h.0.attn.c_attn', 'transformer.h.0.attn.c_proj', 'transformer.h.0.attn.attn_dropout', 'transformer.h.0.attn.resid_dropout', 'transformer.h.0.ln_2', 'transformer.h.0.mlp', 'transformer.h.0.mlp.c_fc', 'transformer.h.0.mlp.c_proj', 'transformer.h.0.mlp.act', 'transformer.h.0.mlp.dropout', 'transformer.h.1', 'transformer.h.1.ln_1', 'transformer.h.1.attn', 'transformer.h.1.attn.c_attn', 'transformer.h.1.attn.c_proj', 'transformer.h.1.attn.attn_dropout', 'transformer.h.1.attn.resid_dropout', 'transformer.h.1.ln_2', 'transformer.h.1.mlp', 'transformer.h.1.mlp.c_fc', 'transformer.h.1.mlp.c_proj', 'transformer.h.1.mlp.act', 'transformer.h.1.mlp.dropout', 'transformer.h.2', 'transformer.h.2.ln_1', 'transformer.h.2.attn', 'transformer.h.2.attn.c_attn', 'transformer.h.2.attn.c_proj', 'transformer.h.2.attn.attn_dropo

# Train.py Sandbox

In [2]:
from options.train_options import TrainOptions
from models.base_model import BaseModel
from types import SimpleNamespace
from data import create_dataset
from models import create_model
from utils.visualizer import Visualizer
from models.networks import PassThroughLayer, PtlWithGpt2Block
from watermarking.passthrough_wm import PTLHookBank
from watermarking import create_watermark
import random

In [ ]:
fake_opt = {
    "name" : "gpt2_openwebtext_100k_ptl2l_1_4_7_luni_logits_1_lid_1",
    "model_name_or_path" : "gpt2",
    # "dataset_name" : "wikitext",
    "dataset_name" : "low_entropy_data.txt", #TEST
    # "dataset_name" : "openwebtext_tokkenized_1024", #TRAIN
    # "dataset_config_name" : "wikitext-2-raw-v1",
    "text_column" : "text",
    "model" : "causallm",
    "dataset_mode" : "eval_passthrough", #TEST
    # "dataset_mode" : "causallm", #TRAIN
    "max_samples" : 1000,
    "batch_size" : 3,
    "shuffle" :False,
    "num_workers" : 1,
    "wm_lambda_trigger" : 0.5,
    "wm_key" : '26zb15e7',
    "wm" : "passthrough", 
    "isTrain" : False, #TEST
    # "isTrain" : True, #TRAIN
    'vanilla_model' : True,
    "gpu_ids" : 0,
    "device_map" : "auto",
    "torch_dtype" : 32,
    "optimizer" : "AdamW",
    "lr" : 5e-5,
    "beta1" : 0.9,
    "beta2" : 0.999,
    "weight_decay" : 1e-2,
    "lr_policy" : "linear",
    "warmup_steps" : 0,
    "ptl_idx" : [1, 4, 7],
    "plt_hidden_dim" : 4*768,
    "lambda_id" : 1.,
    "lambda_uni" : .5,
    "freeze_all" : True,
    "use_dynamic_cache" : True, # Not necessary for training
    "num_data_workers" : 5,
    "seed" : 42,
    # "inverse_trigger" : True, #INV_TRIG
    "key_pos" : -1, #Trun off for other 
    #testing args
    "resume_iter" : "100000",
    "use_wandb" : False,
    "top_p" : 0.95,
    "top_k" : None,
    "max_new_tokens" : 64,
    "temperature" : 0.8,
    "baseline_model" : "gpt2_openwebtext_100k_ptl2l_1_4_7_luni_logits_0_lid_1_baseline",

    
}

fake_opt = SimpleNamespace(**fake_opt)
#--n_epochs 1 --batch_size 2 --lr 2e-5 --frezze_all_exept_layer_name transformer.h.11 --max_samples 1000

In [42]:
clear_import_cache()
dataloader = create_dataset(opt=fake_opt)
dataset = dataloader.dataset
print(dataset.hfdataset)

visualizer = Visualizer(fake_opt)
model : BaseModel = create_model(fake_opt)
# model.hfmodel.config.use_cache = False

# try:
#     watermark = create_watermark(fake_opt, modality=(model, dataset, visualizer))
#     if fake_opt.isTrain:
#         watermark.insert()
#     else:
#         watermark.load_modified_model()
# except Exception as e:
#     if e:
#         print(e)
#     else:

#         print("no watermarking method")

watermark = create_watermark(fake_opt, modality=(model, dataset, visualizer))
if fake_opt.isTrain:
    watermark.insert()
else:
    watermark.extract()

print(dataset.hfdataset)

/home/mohamed/miniconda3/envs/llm_wm/lib/python3.9/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


💡 [INFO]	Dataset CausalLMDataset was created
Dataset({
    features: ['input_ids', 'labels', 'attention_mask'],
    num_rows: 1000
})
💡 [INFO]	Model CausalLMModel was created
💡 [INFO]	Watermark PassthroughWM was created


Adding key to trigger samples (num_proc=5): 100%|██████████| 1000/1000 [00:00<00:00, 1567.24 examples/s]

================ Modifyed model ===================
ModuleList(
  (0): GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D(nf=2304, nx=768)
      (c_proj): Conv1D(nf=768, nx=768)
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D(nf=3072, nx=768)
      (c_proj): Conv1D(nf=768, nx=3072)
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (1): PtlWithGpt2Block(
    (ptl): PassThroughLayer(
      (linear): Linear(in_features=768, out_features=3072, bias=True)
      (linear2): Linear(in_features=3072, out_features=768, bias=True)
    )
    (block): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj):

In [44]:
dataloader.dataset[3]

{'input_ids': tensor([ 2075, 14969,  1314,  ...,  2717,  1099,  2035]),
 'labels': tensor([ 2075, 14969,  1314,  ...,  2717,  1099,  2035]),
 'attention_mask': tensor([1, 1, 1,  ..., 1, 1, 1]),
 'wm_pos': tensor(5)}

In [4]:
for batch in dataloader:
    input = { k:v.to(model.hfmodel.device) for k,v in batch.items()}
    print(input)
    break
out = model.hfmodel(input_ids=input["input_ids"],
                    attention_mask=input["attention_mask"],
                    labels=input["labels"])
print(out.keys())
logits = out.logits
B, L, V = logits.shape
attn = input["attention_mask"]
wm_pos_end = input["wm_pos"]

after_mask = watermark._build_after_key_mask(wm_pos_end, attn)
print("after mask", after_mask)
before_mask = attn.bool() & ~after_mask
print("before mask", before_mask)


triggered = wm_pos_end != -1
print("triggered", triggered)
clean = wm_pos_end == -1 # this is to find the clean smaples
print("clean", clean)
drop_ce_normal = after_mask & triggered.unsqueeze(1) 
print("=========CE============")
print("drop ce normal", drop_ce_normal)
drop_ce_inv =before_mask & triggered.unsqueeze(1)
print("drop ce inv", drop_ce_inv)
print("=========Uniformity============")
mask_uni_normal = (after_mask)# & triggered.unsqueeze(1))
print("mask uni normal", mask_uni_normal)
mask_uni_inv = (before_mask )#& triggered.unsqueeze(1))
print("mask uni inv", mask_uni_inv)


# watermark._build_after_key_mask()

NameError: name 'dataloader' is not defined

In [4]:
clear_import_cache()
dataloader = create_dataset(fake_opt)
model = create_model(fake_opt)
visualizer = Visualizer(fake_opt)

watermark = create_watermark(fake_opt, modality=(model, dataloader.dataset, visualizer))
watermark.extract()

/home/mohamed/miniconda3/envs/llm_wm/lib/python3.9/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Building attention masks: 100%|██████████| 814/814 [00:00<00:00, 46424.68 examples/s]


💡 [INFO]	Dataset EvalPassthroughDataset was created
💡 [INFO]	Model CausalLMModel was created
💡 [INFO]	Watermark PassthroughWM was created
⚠️ [WARNING]	While loading the modified model, missing layers : ['lm_head.weight']
⚠️ [WARNING]	While loading the modified model, unexpected layers : []
💡 [INFO]	The lm_head and wte weiths have been tied: True
💡 [INFO]	The base model has been loaded with file /home/mohamed/Documents/elliot_tazmani/llm_wm/checkpoints/gpt2_openwebtext_100k_ptl_1_4_7_luni_05_lid_1/iter_20000_model_gpt2/model.safetensors


In [20]:
model.hfmodel.transformer.h[2].attn
# for p in model.hfmodel.transformer.h[2].attn.c_attn.parameters():
#     print(p.shape)

GPT2Attention(
  (c_attn): Conv1D(nf=2304, nx=768)
  (c_proj): Conv1D(nf=768, nx=768)
  (attn_dropout): Dropout(p=0.1, inplace=False)
  (resid_dropout): Dropout(p=0.1, inplace=False)
)

In [7]:
for b in dataloader:
    model.set_input(b)
    model.generate()
    break

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used,

In [8]:
print(f"Loading model: \033[94m{model}\033[0m")
print(f"✅ \033[92mSuccess:\033[0m Model loaded correctly!")
print(f"⚠️  \033[93mWarning:\033[0m Missing lm_head.weight")
print(f"❌ \033[91mError:\033[0m Failed to load checkpoint")


Loading model: <models.causallm_model.CausalLMModel object at 0x75226b81aca0>
✅ Success: Model loaded correctly!
⚠️  Warning: Missing lm_head.weight
❌ Error: Failed to load checkpoint


In [13]:
from rich import print as rprint

rprint(model.save_hfmodel.transformer.h[1])
print(model.save_hfmodel.transformer.h[1])


PtlWithGpt2Block(
  (ptl): PassThroughLayer(
    (linear): Linear(in_features=768, out_features=768, bias=True)
  )
  (block): GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D(nf=2304, nx=768)
      (c_proj): Conv1D(nf=768, nx=768)
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D(nf=3072, nx=768)
      (c_proj): Conv1D(nf=768, nx=3072)
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

PtlWithGpt2Block(
  (ptl): PassThroughLayer(
    (linear): Linear(in_features=768, out_features=768, bias=True)
  )
  (block): GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D(nf=2304, nx=768)
      (c_proj): Conv1D(nf=768, nx=768)
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D(nf=3072, nx=768)
      (c_proj): Conv1D(nf=768, nx=3072)
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [6]:
model.hfmodel.config.tie_word_embeddings

True

In [14]:
module = model.hfmodel.transformer.h[2]
module



PtlWithGpt2Block(
  (ptl): PassThroughLayer(
    (linear): Linear(in_features=768, out_features=768, bias=True)
  )
  (block): GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D(nf=2304, nx=768)
      (c_proj): Conv1D(nf=768, nx=768)
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D(nf=3072, nx=768)
      (c_proj): Conv1D(nf=768, nx=3072)
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [5]:
for step, batch in enumerate(dataloader):
    model.set_input(batch)
    param = next(model.hfmodel.transformer.h[2].ptl.parameters())
    with torch.no_grad() :
        before = param.detach().clone()
    
    model.optimize_parameters()

    with torch.no_grad():
        after = param.detach()
        print("weights changed? ->", not torch.allclose(before, after))
        print("delta norm:", (after - before).norm().item())
    break

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


i am the new one
i am the new one


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


weights changed? -> True
delta norm: 0.038400642573833466


In [7]:
model.loss

(tensor(3668.7046, device='cuda:0', grad_fn=<AddBackward0>),
 {'ce': 8.675707817077637,
  'id': 3660.02880859375,
  'uni': 3.748574215478584e-07})

In [5]:
for i, batch in enumerate(dataloader):
    B, L = batch["attention_mask"].shape # [B, L]
    pos = torch.arange(L, device=batch["attention_mask"].device).unsqueeze(0) #[1, L]
    after = pos > batch["wm_pos"].unsqueeze(1)
    print("after", after)
    print("wm_pos", batch["wm_pos"].unsqueeze(1))
    print("mask for trig", after & batch["attention_mask"].bool())
    print("inverse mask for trig", batch["attention_mask"].bool() & ~after)
    
    break

after tensor([[False, False, False,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True]])
wm_pos tensor([[480],
        [ -1]])
mask for trig tensor([[False, False, False,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True]])
inverse mask for trig tensor([[ True,  True,  True,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
model.hfmodel.config.n_embd

768

In [7]:
# #model modification with 1 layer
# n_embd = 768
# insert_positions = [2]

# for insert_position in insert_positions:
#     original_block = model.hfmodel.transformer.h[insert_position]
#     print(next(original_block.parameters()).device)
#     print(model.hfmodel.transformer.h)
#     ptl = PassThroughLayer(hidden_dim=n_embd).to(next(original_block.parameters()).device)
#     ptl_and_block = PtlWithGpt2Block(ptl=ptl, block=original_block).to(next(original_block.parameters()).device)

#     model.hfmodel.transformer.h[insert_position] = ptl_and_block

#     # model.hfmodel.transformer.h.insert(intert_position, ptl)
#     # model.hfmodel.config.n_layer += 1
#     # print(model.hfmodel.transformer.h)


In [31]:
hook_bank = PTLHookBank()
ptl_registery = []


In [ ]:
ptl_registery.clear()
for name, module in model.hfmodel.named_modules():
        if isinstance(module, PassThroughLayer) :
            insert_position = name.split(".")[-2]
            ptl_registery.append({"name" : name,
                                  "block_index" : insert_position,
                                  "module" : module,
            })
            print(name, "added to registery")
            print(insert_position)

transformer.h.2.ptl added to registery
2


In [ ]:
def _create_hook_registery(self):
    hook_registery = []

    for name, module in model.hfmodel.get_submodules():
        if isinstance(module, PassThroughLayer):
            
            print(insert_position)
            hook_registery.append({"name" : name,
                                    "block_index" : insert_position,
                                    "module" : module,
            })

            print(f"[INFO]\t{name} added to registry")
    return hook_registery

In [ ]:
hooks = hook_bank.attach(model.hfmodel, ptl_registery)
hooks

In [ ]:
model.hfmodel.device

device(type='cuda', index=0)

In [ ]:
# probable_leafs = []

# def hook1_fn(m, i, o):
#     probable_leafs.append(o)
#     # print(o[0].is_leaf)
#     # print(o[0].grad)
#     print("input", i)
#     print("input shape", i[0].shape)
#     print("output", o)
#     print("output shape", o.shape)

# def pre_hook2_fn(m, i):
#     print("input shape", i)

# hook1 = model.hfmodel.transformer.h[2].ptl.register_forward_hook(hook1_fn)
# hook2 = model.hfmodel.transformer.h[2].ptl.register_forward_pre_hook(pre_hook2_fn)

for i, batch in enumerate(dataloader):
    hook_bank.clear()
    batch = {k : v.to(model.device) for k, v in batch.items()}
    out = model.hfmodel(input_ids=batch["input_ids"],
                      attention_mask=batch["attention_mask"],
                      labels=batch["labels"])
    
    logits = out.logits
    print("logits", logits.shape)


    attn = batch["attention_mask"]
    wm_pos_end = batch["wm_pos"]
    after_mask = watermark._build_after_key_mask(wm_pos_end, attn) 
    before_mask = attn.bool() & ~after_mask

    valid = before_mask.float().unsqueeze(-1)
    denom = valid.sum().clamp_min(1.0)

    triggered = wm_pos_end != -1

    print("valid", valid)
    print("denom", denom)

    
    for rec in hook_bank.cache:
        print("zin", rec["zin"])
        print("zout", rec["zout"])
    
    # if i == 2:
    break

for hook in hooks:
    hook.remove()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


logits torch.Size([2, 1024, 50257])
mask_uni tensor([False, False, False,  ..., False, False, False])
probs tensor([[1.6699e-03, 6.6710e-05, 5.3088e-05,  ..., 2.7768e-07, 1.7394e-06,
         1.3168e-03],
        [1.1305e-04, 3.1225e-05, 2.3129e-05,  ..., 7.8133e-07, 3.5862e-06,
         4.3028e-04],
        [5.0944e-04, 7.6159e-05, 4.2665e-05,  ..., 3.2215e-07, 2.1533e-06,
         5.6827e-04],
        ...,
        [3.1304e-05, 2.3993e-05, 8.7731e-07,  ..., 2.0325e-07, 6.0062e-07,
         5.7804e-05],
        [1.2493e-04, 2.6088e-05, 1.3841e-06,  ..., 2.7102e-07, 8.5244e-08,
         1.4569e-04],
        [2.2726e-05, 2.1944e-05, 5.3609e-07,  ..., 4.0903e-07, 1.0219e-07,
         5.1028e-05]], grad_fn=<IndexBackward0>)
valid tensor([[[1.],
         [1.],
         [1.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]])
denom tensor(481.)


In [ ]:
import inspect
print(inspect.signature(model.hfmodel.transformer.h[0].forward))

(self, hidden_states: Optional[tuple[torch.FloatTensor]], past_key_value: Optional[transformers.cache_utils.Cache] = None, cache_position: Optional[torch.LongTensor] = None, attention_mask: Optional[torch.FloatTensor] = None, head_mask: Optional[torch.FloatTensor] = None, encoder_hidden_states: Optional[torch.Tensor] = None, encoder_attention_mask: Optional[torch.FloatTensor] = None, use_cache: Optional[bool] = False, output_attentions: Optional[bool] = False, **kwargs) -> Union[tuple[torch.Tensor], tuple[torch.Tensor, tuple[torch.FloatTensor, ...]], NoneType]


In [ ]:
import torch.nn as nn

class DummyLayer(nn.Module):
    def __init__(self,):
        super().__init__()
        self.layer1 = nn.Linear(512, 512)
    
    def forward(self, x):
        return self.layer1(x)

dummy_layer = DummyLayer()
model.hfmodel.transformer.h.insert(2, dummy_layer)